In [454]:
import os
import json
import requests
import re
regex = re.compile(r".+t=([0-9]*h?[0-5]?[0-9]m?[0-5]?[0-9]s)")

from dotenv import load_dotenv
import pandas as pd
from bs4 import BeautifulSoup as bs

from apiclient.discovery import build

In [455]:
API_KEY = os.getenv("API_KEY")
API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = 'v3'
video_id = "3q35rP8AKq4"

In [456]:
youtube = build(
    API_SERVICE_NAME,
    YOUTUBE_API_VERSION,
    developerKey=API_KEY
)

### 動画情報取得
- 動画タイトルや詳細情報など基本情報を取得する

In [457]:
# 動画情報取得
res = youtube.videos().list(
    part = 'snippet,statistics',
    id = video_id
).execute()
title = res['items'][0]['snippet']['title']
description = res['items'][0]['snippet']['description']

In [458]:
res

{'kind': 'youtube#videoListResponse',
 'etag': 'm61VUYbOZG3Q0MnNfqxd4c0OZtM',
 'items': [{'kind': 'youtube#video',
   'etag': 'Lcf4ZEekJiZsBktynl9yGL5nKT8',
   'id': '3q35rP8AKq4',
   'snippet': {'publishedAt': '2014-06-22T10:03:50Z',
    'channelId': 'UCCoHS1W5__eGoaQoQWOwugQ',
    'title': '【作業用BGM】 LaTale BGM集',
    'description': '2013/09/24 作成。\n個人的にお気に入りなBGMの詰め合わせ。\n(日本未実装の曲含みます。)\n作業のお供にどうぞ。\n\n* Track List\n01.MAY be happy\n02.エリアス\n03.エリアス王宮\n04.龍京\n05.アオイチ\n06.枯れ木の森\n07.花の渓谷\n08.雪原\n09.天空豆の樹塔\n10.ジャングル地帯\n11.MAY be happy (summer ver)\n12.オルカ海辺\n13.アクアリウム\n14.エッグエッグ城\n15.砂漠地帯\n16.山岳地帯\n17.桜木の湖\n18.十六夜の間\n19.プロローグ Part2\n20.赤龍の巣\n21.シャングリラ\n22.ピラミッド玄室\n23.厳冬のラビリンス\n24.タイムアタックダンジョン\n25.アビオクラブ\n26.ジョアン・ファームのテーマ\n27.MAY be happy (winter ver)\n28.ビッグアップル\n29.Echochrome Magician\n30.ビフレスト\n31.格納庫\n32.巨大プラント\n33.鋼の都市\n34.ミッドガルド\n35.アトランティス\n36.may be happy forever\n37.ナ・ムーウェンのテーマ\n38.ガマガエル洞窟\n39.ゼナディア廃墟地域\n40.ポウ邸宅\n41.イイダコ神殿\n42.ESP Overload\n43.神秘の森\n44.ギルド連合\n45.エルアノール平原 東部\n46.狐娘寺

In [459]:
title, description

('【作業用BGM】 LaTale BGM集',
 '2013/09/24 作成。\n個人的にお気に入りなBGMの詰め合わせ。\n(日本未実装の曲含みます。)\n作業のお供にどうぞ。\n\n* Track List\n01.MAY be happy\n02.エリアス\n03.エリアス王宮\n04.龍京\n05.アオイチ\n06.枯れ木の森\n07.花の渓谷\n08.雪原\n09.天空豆の樹塔\n10.ジャングル地帯\n11.MAY be happy (summer ver)\n12.オルカ海辺\n13.アクアリウム\n14.エッグエッグ城\n15.砂漠地帯\n16.山岳地帯\n17.桜木の湖\n18.十六夜の間\n19.プロローグ Part2\n20.赤龍の巣\n21.シャングリラ\n22.ピラミッド玄室\n23.厳冬のラビリンス\n24.タイムアタックダンジョン\n25.アビオクラブ\n26.ジョアン・ファームのテーマ\n27.MAY be happy (winter ver)\n28.ビッグアップル\n29.Echochrome Magician\n30.ビフレスト\n31.格納庫\n32.巨大プラント\n33.鋼の都市\n34.ミッドガルド\n35.アトランティス\n36.may be happy forever\n37.ナ・ムーウェンのテーマ\n38.ガマガエル洞窟\n39.ゼナディア廃墟地域\n40.ポウ邸宅\n41.イイダコ神殿\n42.ESP Overload\n43.神秘の森\n44.ギルド連合\n45.エルアノール平原 東部\n46.狐娘寺\n47.虹の釣り堀\n48.タイトル\n49.忘れられた平原\n50.忘れられた花園\n51.イベント2\n52.ソレイユのテーマ\n53.解かれた古の封印ボス戦 デゴス・イフリート\n54.星くずの平原\n55.星のゆりかごボス戦 ルセリア・テセウス\n56.嘆きの海岸\n57.崩壊の塔\n58.崩壊の塔 最上階\n59.飛行艇墜落地帯\n60.漂流都市\n61.漂流都市ボス前\n62.ケネペ\n63.GMイベント')

### コメント一覧から動画時間のコメントを抽出する
- 動画時間のコメントからチャプターコメントを抽出する。
- 最も動画リンクを多くもつコメントが、動画の全チャプターを網羅しているコメントとする。

In [460]:
# コメント一覧取得
res = youtube.commentThreads().list(
    part = 'id,snippet,replies',
    videoId = video_id,
    maxResults = 100,
    order = "relevance"
).execute()

In [461]:
res

{'kind': 'youtube#commentThreadListResponse',
 'etag': 'cLAzgsCLIvWTfepCYusthimXP48',
 'nextPageToken': 'Z2V0X3JhbmtlZF9zdHJlYW1zLS1DdnNEQ0lBRUZSZTMwVGdhOEFNSzZ3TUkyRjhRZ0FRWUJ5TGdBd3VGVEJhVklUZ1RLRjBhNF9OWFhEVDBPNWpLTWNrQ1dyT1d1VVA4QzVMWllVbzJ5VUtUU2dqWVpjU1VKSUNkdDh4c21sc3kxQWExMklxWjh2UXNMb29TWXRKWXhsUXB1M3d6aUo3TExib2RsMnJRR2twWVVORzBzelhrV3BUQ1JRZW15SkZRUjZaVEVtSEpDT00yT0dzZ0ZSQUZVTmxFRUs3X1NVR3pMcDBoNDV0THRMcHRrUmJhNnNCeGNnZHJVZ2NtT2ZPeVVoY29jZ1BXZHRocVR6b3hFOWFhcVVpSUNEcG9vRll1ckRtUER5Tmptc3FWZ3NxQWJnWW1OQm9iQjJlcEVJNm9ac1FlX2k0Skpxd2pnVTRtUmxBa3J1LTA2NE8xZldRWGdVRkVJY2lNYmVDNTBEeEd0VWYwYWItWEtKcVdEakxaU2R4V0RZcEsxUE04Y2xJcFBBaU9WU0F5cUJJSWJkaUdHRWt1TVNyVGNHSXM3RlBFU1VBR1dVUENoQXN5OEFUNjR3WnZXRVV4WTFSRUYyZ01wWFpDUUM4U0puUlpwVVdua1J3aEttUWZQUFZteTNOMDdBaURpTF9ZSkNfT3FpV2h2VGpnbkMtbnRaVmhZUFZ4YzRLc3NGMmU1M3FWRFRLeHFRWVFpdzl1bW04N3dWa2tSQ0w4cVpWZWlyTExvcnNDMGpIcFhrQXZxZDBiX2l2SHBfVFprZ1lQdzF1QkxpUjFJWldJMUFsUmRQRXJnNnpBWkNpTXJkbFU3UEllT3RFSHF1YzROMWk2TnlLSmtPV2FSS0pBbDBsZVFFSnFZcGI

In [462]:
# 一番人気のコメントをチャプターコメントとみなす
# TODO: 余裕があれば、video_idを一番多く含むコメントをチャプターコメントとみなす。
original = res["items"][0]["snippet"]["topLevelComment"]["snippet"]["textOriginal"]
display = res["items"][0]["snippet"]["topLevelComment"]["snippet"]["textDisplay"]

time_name_dic = {}
chapters = []

for line in original.splitlines():
    match = re.match(r"([0-9]*:?[0-5]?[0-9]:?[0-5]?[0-9])(.*)[\r?\n|\r\n?]*", line)
    if match:
        if len(match.groups()) == 2:
            time, name = match.groups()[0], match.groups()[1]
            time_name_dic[time] = name.strip()

soup = bs(display, "html.parser")

from datetime import datetime as dt
for tag in soup.find_all("a"):
    link = tag["href"]
    time = tag.string
    match = re.match(regex, link)
    if match:
        seconds = match.groups()[0]
        chapters.append([seconds, time, time_name_dic.get(time), link])

df_chapter = pd.DataFrame(chapters, columns=["seconds", "time", "name", "link"])

# linkの中の秒数を抜き出して、timedeltaに変換して、秒数に変換する
df_chapter["seconds"] = pd.to_timedelta(df_chapter["seconds"]).dt.total_seconds().astype(int)
df_chapter = df_chapter.set_index("seconds")
df_chapter

,time,name,link
seconds,,,
1,0:01,MAY be happy,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
81,1:21,エリアス,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
204,3:24,エリアス王宮,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
359,5:59,龍京,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
474,7:54,アオイチ,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
...,...,...,...
7396,2:03:16,飛行艇墜落地帯,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
7520,2:05:20,漂流都市,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
7644,2:07:24,漂流都市ボス前,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...


### 動画のheatmapデータを取得する

In [463]:
video_id = "3q35rP8AKq4"
res = requests.get(f"https://www.youtube.com/watch?v={video_id}")

soup = bs(res.text, "html.parser")
data = re.search(r"var ytInitialData = ({.*?});", soup.prettify()).group(1)
data = json.loads(data)
heatmap = data['playerOverlays']['playerOverlayRenderer']['decoratedPlayerBarRenderer']['decoratedPlayerBarRenderer']['playerBar']['multiMarkersPlayerBarRenderer']['markersMap'][0]["value"]["heatmap"]["heatmapRenderer"]["heatMarkers"]
df_heatmap = pd.json_normalize(heatmap)
df_heatmap["seconds"] = round(df_heatmap["heatMarkerRenderer.timeRangeStartMillis"]/1000).astype(int)
df_heatmap = df_heatmap.set_index("seconds")
df_heatmap["time"] = pd.to_datetime(df_heatmap["heatMarkerRenderer.timeRangeStartMillis"], unit="ms").dt.strftime("%H:%M:%S")
df_heatmap

,heatMarkerRenderer.timeRangeStartMillis,heatMarkerRenderer.markerDurationMillis,heatMarkerRenderer.heatMarkerIntensityScoreNormalized,time
seconds,,,,
0,0,79370,1.000000,00:00:00
79,79370,79370,0.231870,00:01:19
159,158740,79370,0.196624,00:02:38
238,238110,79370,0.089847,00:03:58
317,317480,79370,0.461102,00:05:17
...,...,...,...,...
7540,7540150,79370,0.021168,02:05:40
7620,7619520,79370,0.118200,02:06:59
7699,7698890,79370,0.114757,02:08:18


### ヒートマップスコア上位と一番近い時間の動画チャプターを抜き出す

In [476]:
# 0秒目は必ずScore1.0になるので、除く
n = 10
top_n = df_heatmap.sort_values("heatMarkerRenderer.heatMarkerIntensityScoreNormalized", ascending=False)[1:].head(n)
top_chapter_indices = df_chapter.index.get_indexer(top_n.index, method="nearest")

df_chapter.iloc[top_chapter_indices].drop_duplicates()

top_n.index Index([6588, 2857, 2778, 317, 2460, 2937, 2699, 5477, 6667, 3016], dtype='int32', name='seconds')


,time,name,link
seconds,,,
6589,1:49:49,解かれた古の封印ボス戦 デゴス・イフリート,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
2856,47:36,厳冬のラビリンス,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
2735,45:35,ピラミッド玄室,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
359,5:59,龍京,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
2491,41:31,赤龍の巣,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
2977,49:37,タイムアタックダンジョン,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
5481,1:31:21,エルアノール平原 東部,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
6713,1:51:53,星くずの平原,https://www.youtube.com/watch?v=3q35rP8AKq4&t=...
